## mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

## Import

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import cv2
import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import style
from PIL import Image
import glob
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam


## Visual and Augment

In [ ]:
# List all data
members = ["Oak","Pat","Pookkie","Praewa","Tup"]
image_dir = "/content/drive/MyDrive/DeepLearning-Project/Train_Splitted/train" # Path to dataset

total_images = 0

for member in members:
    count = len([f for f in os.listdir(os.path.join(image_dir, member))
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    print(f"{member}'s images: {count}")
    total_images += count

print(f"Total images: {total_images}")

In [ ]:
# Show data
style.use('dark_background')
plt.figure(figsize=(20, 15))
for row, member in enumerate(members) :
  image_folder = os.path.join(image_dir, member)
  image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
  sample_images = random.sample(image_files, min(5, len(image_files)))
  for col, filename in enumerate(sample_images):
      img_path = os.path.join(image_folder, filename)
      img = cv2.imread(img_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert color for Matplotlib

      plt.subplot(len(members), 5, row * 5 + col + 1)
      plt.imshow(img)
      plt.axis("off")
      plt.title(member if col == 2 else "", fontsize=25,color="yellow",fontweight="bold")

plt.tight_layout()
plt.show()


In [ ]:
def augment_pipeline(image_dir, members, augmentation_factor=3, img_size=(256, 256)):
    """
    Loads images, applies augmentation, and returns them without saving.

    Parameters:
        image_dir (str): Path to the original images.
        members (list): List of subdirectories (categories).
        augmentation_factor (int): Number of augmented images per original image.
        img_size (tuple): Target image size (width, height).

    Returns:
        dict: Dictionary with member names as keys and lists of augmented images as values.
    """
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    augmented_data = {}

    for member in members:
        input_folder = os.path.join(image_dir, member)
        image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        print(f"Processing {member}: Found {len(image_files)} images")
        augmented_data[member] = []

        for filename in image_files:
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            img = cv2.resize(img, img_size)
            img = np.expand_dims(img, axis=0)

            i = 0
            for batch in datagen.flow(img, batch_size=1):
                augmented_data[member].append(batch[0].astype(np.uint8))
                i += 1
                if i >= augmentation_factor:
                    break

        print(f"{member}: {len(augmented_data[member])} augmented images generated")

    return augmented_data


augmented_images = augment_pipeline(image_dir, members, augmentation_factor=3, img_size=(256, 256))
print("Augmentation complete.")

In [ ]:
# Show Augment data
style.use('dark_background')
plt.figure(figsize=(20, 15))

for row, (member, images) in enumerate(augmented_images.items()):
    sample_images = images[:5]

    for col, img in enumerate(sample_images):
        plt.subplot(len(members), 5, row * 5 + col + 1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(member if col == 2 else "", fontsize=25,color="yellow",fontweight="bold")

plt.tight_layout()
plt.show()


In [ ]:
# Save Augment to folder
augmented_images_path = "/content/drive/MyDrive/DeepLearning-Project/Augmented_train"

for member, images in augmented_images.items():
    member_path = os.path.join(augmented_images_path, member)
    os.makedirs(member_path, exist_ok=True)  # Create the member directory if it doesn't exist
    for i, img in enumerate(images):
        filename = os.path.join(member_path, f"augmented_{member}_{i}.jpg")
        cv2.imwrite(filename, img)

## Training

In [ ]:
img_height, img_width = 224, 224
batch_size = 32

train_dir = "/content/drive/MyDrive/DeepLearning-Project/Augmented_train"
test_dir = "/content/drive/MyDrive/DeepLearning-Project/Train_Splitted/test"

class_names = os.listdir(train_dir)
image_paths = []
labels = []
test_image_paths = []
test_labels = []

for class_index, class_name in enumerate(class_names):
    class_path = os.path.join(train_dir, class_name)
    for img in os.listdir(class_path):
        image_paths.append(os.path.join(class_path, img))
        labels.append(class_index)

for class_index, class_name in enumerate(class_names):
    class_path = os.path.join(test_dir, class_name)
    for img in os.listdir(class_path):
        test_image_paths.append(os.path.join(class_path, img))
        test_labels.append(class_index)

print(class_names)
print(len(image_paths))
print(len(labels))
print(len(test_image_paths))
print(len(test_labels))

In [ ]:
# Using VGG16 for Base model and add 2 layer for training Data

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

base_model.trainable = False

model = Sequential([
  base_model,
  GlobalAveragePooling2D(),  # To reduce spatial dimensions
  Dense(1024, activation='relu'),  # A dense layer for additional learning
  Dense(5, activation='softmax')  # Output layer with 5 classes
])

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Convert to DataFrame
image_data = pd.DataFrame({"filename": image_paths, "label": labels})
image_data
image_data["label"] = image_data["label"].astype(str)
image_data.dtypes

In [ ]:
# Convert to DataFrame
test_image_data = pd.DataFrame({"filename": test_image_paths, "label": test_labels})
test_image_data
test_image_data["label"] = test_image_data["label"].astype(str)
test_image_data.dtypes

In [ ]:
train_df, val_df = train_test_split(image_data, test_size=0.1, stratify=image_data['label'], random_state=42)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_image_data,
    x_col='filename',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
NUM_EPOCHS = 2

history = model.fit(
  train_generator,
  validation_data=val_generator,
  epochs=NUM_EPOCHS,
  verbose=1
)

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

## Testing

In [ ]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

In [ ]:
img_path = '/content/drive/MyDrive/DeepLearning-Project/Train_Splitted/test/Pookkie/IMG_4227.JPG' # Testing Image path
preprocessed_image = preprocess_image(img_path)
prediction = model.predict(preprocessed_image)
predicted_class_index = np.argmax(prediction, axis=1)

class_labels = ['Oak', 'Pat', 'Pookkie', 'Praewa', 'Tup']
print(predicted_class_index)
predicted_class_label = class_labels[predicted_class_index[0]]
print(f"Predicted Class: {predicted_class_label}")

## Save Model

In [ ]:
model_save_path = '/content/drive/MyDrive/DeepLearning-Project/saved_models'

os.makedirs(model_save_path, exist_ok=True)

model.save(os.path.join(model_save_path, f"model_Augment_2_2.h5"))

## Verify Model

In [ ]:
# Load the saved model
model_path = '/content/drive/MyDrive/DeepLearning-Project/saved_models/model_Augment_2.h5'
model = tf.keras.models.load_model(model_path)

In [ ]:
def predict_images_from_path(image_dir, model):
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
    predictions = []
    for image_file in image_files:
        img_path = os.path.join(image_dir, image_file)
        Image.open(img_path).verify()
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction, axis=1)
        class_labels = ['Oak', 'Pat', 'Pookkie', 'Praewa', 'Tup']
        predicted_class_label = class_labels[predicted_class_index[0]]
        predictions.append((image_file, predicted_class_label))

    return predictions

In [ ]:
image_directory = '/content/drive/MyDrive/DeepLearning-Project/Train_Splitted/test/Tup/'
results = predict_images_from_path(image_directory, model)

In [ ]:
correct_prediction = 0
all_prediction = 0

for image_file, predicted_label in results:
    print(f"Image: {image_file}, Predicted Label: {predicted_label}")
    if predicted_label == 'Tup':
      correct_prediction += 1
    all_prediction += 1

In [ ]:
print(f"Correct prediction : {correct_prediction} from {all_prediction}")